<a href="https://colab.research.google.com/github/janeshnanda2005/Datasets/blob/main/quickdrawclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data

In [2]:
cd data

/content/data


In [4]:
!wget -P data https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget -P data https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy
!wget -P data https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/dog.npy

--2025-02-08 04:26:10--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.207, 142.251.12.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113462128 (108M) [application/octet-stream]
Saving to: ‘data/apple.npy’

apple.npy           100%[===================>] 108.21M  23.7MB/s    in 5.8s    

2025-02-08 04:26:16 (18.8 MB/s) - ‘data/apple.npy’ saved [113462128/113462128]

--2025-02-08 04:26:16--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.207, 142.251.12.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96590448 (92M) [application/octet-

In [23]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load datasets
def load_data(categories, data_dir="data", samples_per_class=10000):
    X, y = [], []

    for label, category in enumerate(categories):
        file_path = f"{data_dir}/{category}.npy"
        data = np.load(file_path)[:samples_per_class]  # Limit samples per class
        X.append(data)
        y.extend([label] * len(data))

    X = np.concatenate(X, axis=0).reshape(-1, 28, 28, 1).astype(np.float32)
    X /= 255.0
    y = np.array(y)

    return train_test_split(X, y, test_size=0.2, random_state=42)

# Define categories
categories = ["apple", "cat", "dog"]
X_train, X_test, y_train, y_test = load_data(categories)

# Convert labels to one-hot encoding
num_classes = len(categories)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")


Training data shape: (24000, 28, 28, 1), Labels shape: (24000, 3)


In [19]:
import tensorflow as tf

# Define CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])


model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])


model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)                    │ (None, 26, 26, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 11, 11, 16)          │           9,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 5, 5, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          51,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 61,587 (240.57 KB)

 Trainable params: 61,587 (240.57 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the CNN model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=128,
    validation_data=(X_test, y_test)
)


Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.7526 - loss: 0.5645 - val_accuracy: 0.8823 - val_loss: 0.3021
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8773 - loss: 0.3018 - val_accuracy: 0.8898 - val_loss: 0.2752
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8927 - loss: 0.2705 - val_accuracy: 0.8912 - val_loss: 0.2667
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8951 - loss: 0.2500 - val_accuracy: 0.8945 - val_loss: 0.2515
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9036 - loss: 0.2372 - val_accuracy: 0.9010 - val_loss: 0.2457
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9040 - loss: 0.2272 - val_accuracy: 0.9043 - val_loss: 0.2398
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9163 - loss: 0.2080 - val_accuracy: 0.9012 - val_loss: 0.2492
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9190 - loss: 0.1931 - val_accuracy: 0

In [21]:
# Evaluate performance
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9005 - loss: 0.2439
Test Accuracy: 0.9055


In [22]:
model.save("model.h5")

In [25]:
from google.colab import files
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Upload image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Load model
model = load_model("/content/data/model.h5")

# Preprocess image
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype(np.float32) / 255.0
    img = img.reshape(1, 28, 28, 1)
    return img

input_image = preprocess_image(image_path)

# Predict
predictions = model.predict(input_image)
predicted_label = np.argmax(predictions)

# Define categories
categories = ["apple", "cat", "dog"]
emoji_mapping = {"apple": "🍎", "cat": "🐱", "dog": "🐶"}

# Display result
predicted_category = categories[predicted_label]
predicted_emoji = emoji_mapping[predicted_category]

print(f"Recognized Drawing: {predicted_category} {predicted_emoji}")


Saving apple.png to apple (1).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
Recognized Drawing: apple 🍎
